In [14]:
!pip install geemap



# IMPORTING LIBRARIES & STARTING EARTH ENGINE

In [15]:
# Importing the necessary libraries for reading and displaying the image.
# raterio to handle raster data, and maplotlib for visualizing it
%pip install rasterio
from matplotlib import pyplot as plt


Note: you may need to restart the kernel to use updated packages.


In [16]:
import ee
import geemap
import json
import os
import rasterio as rio

# Load the OAuth 2.0 client credentials
client_secret_file = r'C:\Users\Neo\Desktop\FYP\Flood-Detection-System-FYP\client_secret_312891332570-i9hhj74hqa1esuptbbrt60mrkqrq5rhm.apps.googleusercontent.com (1).json'

# Authenticate and initialize Earth Engine
ee.Authenticate()
try:
    ee.Initialize(project='atu-fyp')
    print("Earth Engine initialized successfully.")
except ee.EEException as e:
    print(f"Failed to initialize Earth Engine: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

# Creates the interactive map shown in the next cell
Map = geemap.Map()

# Displaying the map
Map

Earth Engine initialized successfully.


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## CHOOSING THE GEOGRAPHIC SPACE ( Cork )
According to 
The irish times: https://www.irishtimes.com/ireland/2023/10/19/cork-flooding-floods-in-co-cork-absolutely-devastating-as-safety-warning-issued-to-motorists/
Picked the area that got more affected with the floods of 2023


In [17]:
# Defines the area of interest in a rectangular shape 
area_of_interest = ee.Geometry.Polygon([ # This polygon is defined using four sets of coodinates , each representing a corner of the rectangle
        [[-8.570156845611686, 52.00904254772663], # Top-left corner
         [-8.570156845611686, 51.7292195887807], # Bottom-left corner
         [-7.959042343658562, 51.7292195887807], # Bottom-right corner
         [-7.959042343658562, 52.00904254772663]] # Top-right corner these coordinates are in [longitude, latitude] format, following the convention for geographic coordinates in Earth Engine
    ]).buffer(5000) # Adding a buffer to expand the area only for data analyses process further in the project

# Initialize the map
Map = geemap.Map(center=[51.8691, -8.2646], zoom=10)  # Adjusts center and zoom as needed

# Adds the area of interest to the map
Map.addLayer(area_of_interest, {'color': 'red'}, 'Area of Interest')


In [18]:
# Define the time range
time_start = '2023-01-01'
time_end = '2024-01-01'

# Load the image collection and apply filters
area_of_interest_before = (ee.ImageCollection('COPERNICUS/S1_GRD')
          .filterDate(time_start, time_end)
          .filterBounds(area_of_interest)
          .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
          .filter(ee.Filter.eq('instrumentMode', 'IW'))
          .select('VV')
          .min())

# Print the filtered image information
print(area_of_interest_before.getInfo())

# Add the layer to the map with the clipped geometry
Map.addLayer(area_of_interest_before.clip(area_of_interest), {}, 'Before', False)
Map.centerObject(area_of_interest, 10)

# Display the map
Map

{'type': 'Image', 'bands': [{'id': 'VV', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


Map(center=[51.8693369656094, -8.264599594970292], controls=(WidgetControl(options=['position', 'transparent_b…

## DATE RANGE AND FILTERING SENTINEL COLLECTION

In [19]:
# Time range from 2015 to 2023
start_date = '2015-01-01'
end_date = '2023-12-31'

# Fetch Landsat images for the defined area and time range, and applying a cloud cover filter
sentinel_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
                     .filterBounds(area_of_interest) \
                     .filterDate(start_date, end_date)

## RETRIEVING AND DISPLAYING FIRST IMAGE METADATA

In [20]:
# Get the first image from the collection
first_image = sentinel_collection.first()

# Print image information
print(first_image.getInfo())

{'type': 'Image', 'bands': [{'id': 'HH', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [12688, 12341], 'crs': 'EPSG:32630', 'crs_transform': [40, 0, 75010.67228783463, 0, -40, 5920472.687766667]}, {'id': 'HV', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [12688, 12341], 'crs': 'EPSG:32630', 'crs_transform': [40, 0, 75010.67228783463, 0, -40, 5920472.687766667]}, {'id': 'angle', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [21, 22], 'crs': 'EPSG:32630', 'crs_transform': [20481.23323360778, -3402.2406153591, 172027.2353513519, 3343.503048089333, 20199.185939010233, 5436476.418459804]}], 'version': 1739793628475975, 'id': 'COPERNICUS/S1_GRD/S1A_EW_GRDM_1SDH_20150104T180509_20150104T180610_004025_004D9B_C7FC', 'properties': {'SNAP_Graph_Processing_Framework_GPF_vers': '6.0.4', 'SLC_Processing_facility_org': 'Airbus DS', 'SLC_Processing_facility_country': 'United Kingdom', 'GRD_Post_Processing_facility_org': '

## EXPORT FIRST IMAGE TO GOOGLE DRIVE

In [21]:
# Export the image to Google Drive (you'll need to have Google Drive connected)
task = ee.batch.Export.image.toDrive(
    image=first_image,
    description='Sentinel_Composite',
    folder='EarthEngineImages',
    scale=30,
    region=area_of_interest
)
task.start()

print("Image export task completed...")


Image export task completed...


## CONVERTING IMAGE TO Uint16 FORMAT IN GEE
images have different data types across bands. I was having an error saying that i couldnt display the image in ( "Map" ) from gee this is often required for further image processing and exporting tasks.

In [22]:
# Convert the image to Uint16 to standardize data types across all bands
first_image = first_image.toUint16()

# Exporting task to export the image to google drive cloud services
task = ee.batch.Export.image.toDrive(
    image=first_image,
    description='Sentinel_Composite',
    folder='EarthEngineImages',
    scale=30,
    region=area_of_interest
)
# This task runs asynchronously and we can monitor its progress on GEE.
task.start()

print("Export task started...")


Export task started...


In [23]:
#Provides information wether the export is still in progress,completed, or failed.
task.status()

{'state': 'READY',
 'description': 'Sentinel_Composite',
 'priority': 100,
 'creation_timestamp_ms': 1739794583436,
 'update_timestamp_ms': 1739794583436,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'MDSIUWY5TPKVEOPUH252E35P',
 'name': 'projects/atu-fyp/operations/MDSIUWY5TPKVEOPUH252E35P'}

DISPLAYING IMAGE

In [24]:
# Open the image from my desktop
with rasterio.open(r"C:\Users\Neo\Desktop\FYP\geeimages\Sentinel_Composite.tif") as src:
    # reading the image
    image = src.read(1)  # Read the first band

# Displaying the image using a grayscale colour map.
plt.imshow(image, cmap='gray')
plt.colorbar()
plt.title('Sentinel Composite image')
plt.show()



NameError: name 'rasterio' is not defined

In [ ]:
# Open the image from my desktop
with rasterio.open(r"C:\Users\Neo\Desktop\FYP\geeimages\Sentinel_Composite2.tif") as src:
    # reading the image
    image = src.read(1)  # Read the first band

# Displaying the image using a grayscale colour map.
plt.imshow(image, cmap='gray')
plt.colorbar()
plt.title('Landsat image')
plt.show()



In [ ]:
# Load the Sentinel-1 Image Collection from Google Earth Engine.
# This collection contains synthetic aperture radar (SAR) data captured by Sentinel-1 satellites.
sentinel_collection = ee.ImageCollection('COPERNICUS/S1_GRD')

# Define the date range for the analysis.
# 'start_date' is the beginning of the time period, and 'end_date' is the end of the time period.
start_date = '2015-01-01'
end_date = '2023-12-31'

# Filter the Sentinel-1 Image Collection by:
# 1. Spatial bounds (area_of_interest): Only images within the specified area are included.
# 2. Temporal bounds: Only images captured within the date range (start_date to end_date) are included.
filtered_collection = sentinel_collection.filterBounds(area_of_interest).filterDate(start_date, end_date)

# Retrieve the first image from the filtered collection.
# This is useful for inspecting and understanding the structure or metadata of a sample image from the dataset.
first_image = filtered_collection.first()

# Print the metadata of the first image to the console.
# This metadata includes information such as image properties, bands, and acquisition date.
print(first_image.getInfo())


# The image is clipped to the area of interest, and the 'VV' band is visualized with specific display settings:
# The layer is named 'First Image'.
Map.addLayer(area_of_interest_before.clip(area_of_interest), {'bands': ['VV'], 'min': -20, 'max': 0}, 'Gray scale')

# Center the map display to the area of interest for better visualization.
Map.centerObject(area_of_interest)

# Display the map. This will show all added layers and features in the Google Earth Engine interface.
Map


When the code is executed thid displays:
A grayscale map layer representing the area_of_interest_before image (radar data clipped to the specified region).
The intensity in the VV band, where darker regions might indicate water or vegetation, and brighter regions might represent urban areas or bare ground.

In [ ]:
# Load and filter Sentinel-2 Image Collection (if not done in previous steps)
sentinel2_collection = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(area_of_interest) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

# Function to calculate NDWI for a given image
def add_ndwi(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')  # Calculate NDWI
    return image.addBands(ndwi)  # Add NDWI as a new band

# Map the function over the collection
ndwi_collection = sentinel2_collection.map(add_ndwi)

# Define threshold and create a binary water mask
def apply_ndwi_threshold(image):
    water_mask = image.select('NDWI').gt(0.3)  # Apply threshold
    return image.addBands(water_mask.rename('WaterMask'))  # Add mask as new band

# Apply the thresholding function to the collection
water_masked_collection = ndwi_collection.map(apply_ndwi_threshold)

# Select an example image to visualize NDWI and water mask
example_image = water_masked_collection.first().clip(area_of_interest)

# Visualization parameters for NDWI and water mask
ndwi_viz_params = {'bands': ['NDWI'], 'min': -1, 'max': 1, 'palette': ['white', 'blue']}
water_mask_viz_params = {'bands': ['WaterMask'], 'min': 0, 'max': 1, 'palette': ['white', 'blue']}

# Add the layers to the map
Map.addLayer(example_image.clip(area_of_interest), ndwi_viz_params, 'NDWI')
Map.addLayer(example_image.clip(area_of_interest), water_mask_viz_params, 'Water Mask')
Map.centerObject(area_of_interest)
Map
